In [18]:
#Librerias que se utilizaran
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import duckdb
import glob

# Librerias astrofisica
# =============================================================================
from astroquery.mast import Observations
import lightkurve as lk
from astropy.time import Time

# Series de tiempo
# ===============================================================================
import pmdarima

# Configuración para las series
# ================================================================================
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import pacf
import statsmodels.api as sm

# Configuración para machine learning
# ================================================================================
import tensorflow as tf
import tensorflow.keras as keras
from pandas.core.frame import DataFrame
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

con = duckdb.connect(database=':memory:')

2023-10-30 17:19:45.587918: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 17:19:46.030147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9241] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 17:19:46.030252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 17:19:46.118674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1516] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 17:19:46.286548: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 17:19:46.290264: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
#Urls Pater
url_lc = "/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/Curvas_de_luz/"
url    = "/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/"

In [ ]:
#Url Ari 
url_lc = "C:/Users/ASUS/OneDrive/Documentos/2023-2/Inteligencia/Proyecto/Exoplanet_IA/Curvas_de_luz/"
url    = "C:/Users/ASUS/OneDrive/Documentos/2023-2/Inteligencia/Proyecto/Exoplanet_IA/"

## Conto de estrellas por clase

In [7]:
Carac_Stars = pd.read_csv('Carac_Stars.csv')
Carac_Stars

,Kepler_ID,Metalicidad,Clasificacion,incert_clas
0,10797460,0.136,CANDIDATE,1.000
1,10811496,-0.127,FALSE POSITIVE,0.000
2,10848459,-0.522,FALSE POSITIVE,0.000
3,10854555,0.081,CANDIDATE,1.000
4,10872983,-0.067,CANDIDATE,0.992
...,...,...,...,...
7677,10128825,-0.046,CANDIDATE,0.497
7678,10147276,-0.038,FALSE POSITIVE,0.021
7679,10156110,0.082,FALSE POSITIVE,0.000
7680,8460634,-0.556,FALSE POSITIVE,0.000


In [8]:
con.execute("""select Clasificacion, count(*)
               from Carac_Stars
               group by Clasificacion""").df()

,Clasificacion,count_star()
0,CANDIDATE,3246
1,FALSE POSITIVE,4436


## Descripción de las series de tiempo

### Registros en el tiempo

In [10]:
arch = os.listdir(url_lc)
L_Str = []
series = np.zeros(len(arch))
for Str in arch:
    i = arch.index(Str)
    str_id = int(Str[3:-4])
    L_Str.append(str_id)
    
    serie = pd.read_csv(url_lc+Str)
    series[i] = serie

    

In [16]:
serie = pd.read_csv(url_lc+arch[0])
serie

,time,flux,flux_err
0,1559.225877,NaN,NaN
1,1559.246311,13539.965,4.322397
2,1559.266745,13553.622,4.324529
3,1559.287179,13546.764,4.322358
4,1559.307613,13551.458,4.322149
...,...,...,...
1281,1590.919400,13503.387,4.291182
1282,1590.939834,13495.783,4.287817
1283,1590.960268,13494.557,4.291154
1284,1590.980703,13490.738,4.290288


In [36]:
#Los datos se encuentran en bkjd (Barycentric Kepler Julian Date)
time = Time(serie['time'], format='bkjd')  # Crear un objeto Time de astropy
datetime_time = time.datetime  # Convertir a datetime
help(datetime_time[0])
#time

Help on datetime object:

class datetime(date)
 |  datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])
 |  
 |  The year, month and day arguments are required. tzinfo may be None, or an
 |  instance of a tzinfo subclass. The remaining arguments may be ints.
 |  
 |  Method resolution order:
 |      datetime
 |      date
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __radd__(self, value, /)
 |      Retur